<a href="https://colab.research.google.com/github/skyprince999/100-Days-Of-ML/blob/master/Transfer_Learning_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html